<a href="https://colab.research.google.com/github/dwwatson1/coronavirus_pandemic_playbook/blob/michael_work/cdc_main_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.1.2'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/graph

In [29]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrameBasics").getOrCreate()

In [30]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://cdc-main-database.s3.amazonaws.com/COVID-19_Case_Surveillance_Public_Use_Data_with_Geography.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("COVID-19_Case_Surveillance_Public_Use_Data_with_Geography.csv"), header=True)

In [31]:
df.show()

+----------+---------+---------------+----------+----------------+--------------+------+-------+---------+-------------------------------+-------------------+--------------------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|case_month|res_state|state_fips_code|res_county|county_fips_code|     age_group|   sex|   race|ethnicity|case_positive_specimen_interval|case_onset_interval|             process|exposure_yn|      current_status|symptom_status|hosp_yn| icu_yn|death_yn|underlying_conditions_yn|
+----------+---------+---------------+----------+----------------+--------------+------+-------+---------+-------------------------------+-------------------+--------------------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|   2020-08|       KY|             21|   GRAYSON|           21085|  0 - 17 years|    NA|     NA|       NA|                           null|                  0|        

In [32]:
# Filter dates between March 2020 and December 2020
df = df.filter((df.case_month >= '2020-03') & (df.case_month <= '2020-12'))

In [33]:
df.show()

+----------+---------+---------------+----------+----------------+--------------+------+-------+---------+-------------------------------+-------------------+-------------------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|case_month|res_state|state_fips_code|res_county|county_fips_code|     age_group|   sex|   race|ethnicity|case_positive_specimen_interval|case_onset_interval|            process|exposure_yn|      current_status|symptom_status|hosp_yn| icu_yn|death_yn|underlying_conditions_yn|
+----------+---------+---------------+----------+----------------+--------------+------+-------+---------+-------------------------------+-------------------+-------------------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|   2020-08|       KY|             21|   GRAYSON|           21085|  0 - 17 years|    NA|     NA|       NA|                           null|                  0|           

In [34]:
# Drop unnecessary columns
df = df.drop('case_month', 'state_fips_code', 'res_county', 'county_fips_code', 'case_positive_specimen_interval', 'case_onset_interval')

In [35]:
df.show()

+---------+--------------+------+-------+---------+-------------------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|res_state|     age_group|   sex|   race|ethnicity|            process|exposure_yn|      current_status|symptom_status|hosp_yn| icu_yn|death_yn|underlying_conditions_yn|
+---------+--------------+------+-------+---------+-------------------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|       KY|  0 - 17 years|    NA|     NA|       NA|            Missing|    Missing|Laboratory-confir...|   Symptomatic|     No|Missing|      No|                    null|
|       GA|  0 - 17 years|Female|Unknown|  Missing|            Missing|    Missing|Laboratory-confir...|       Missing|Missing|Missing| Missing|                    null|
|       PA|  0 - 17 years|Female|     NA|       NA|            Missing|        Yes|Laboratory-confir...|   Symptomatic|     No|     No|      NA|      

In [36]:
df.groupby('res_state').count().orderBy('count', ascending=False).show()

+---------+-------+
|res_state|  count|
+---------+-------+
|       CA|2517956|
|       NY|1242822|
|       IL|1030427|
|       FL| 858632|
|       OH| 714699|
|       PA| 666585|
|       TN| 596611|
|       NC| 589914|
|       AZ| 579023|
|       NJ| 552458|
|       GA| 516835|
|       IN| 515233|
|       WI| 440126|
|       MN| 429299|
|       MA| 386352|
|       VA| 369707|
|       AL| 355128|
|       SC| 331485|
|       IA| 327246|
|       MI| 326148|
+---------+-------+
only showing top 20 rows



In [37]:
df.groupby('age_group').count().orderBy('count', ascending=False).show()

+--------------+-------+
|     age_group|  count|
+--------------+-------+
|18 to 49 years|9066479|
|50 to 64 years|3422181|
|     65+ years|2401830|
|  0 - 17 years|1754297|
|            NA| 188666|
|       Missing|  96772|
+--------------+-------+



In [38]:
df.groupby('sex').count().orderBy('count', ascending=False).show()

+-------+-------+
|    sex|  count|
+-------+-------+
| Female|8550138|
|   Male|7734161|
|     NA| 514000|
|Unknown| 112611|
|Missing|  19315|
+-------+-------+



In [39]:
df.groupby('race').count().orderBy('count', ascending=False).show()

+--------------------+-------+
|                race|  count|
+--------------------+-------+
|               White|7250574|
|                  NA|2886927|
|             Unknown|2772223|
|             Missing|1509396|
|               Black|1252891|
|      Multiple/Other| 806242|
|               Asian| 303508|
|American Indian/A...| 124836|
|Native Hawaiian/O...|  23628|
+--------------------+-------+



In [40]:
df.groupby('ethnicity').count().orderBy('count', ascending=False).show()

+-------------------+-------+
|          ethnicity|  count|
+-------------------+-------+
|Non-Hispanic/Latino|6922096|
|            Unknown|3751967|
|                 NA|3471207|
|    Hispanic/Latino|1574833|
|            Missing|1210122|
+-------------------+-------+



In [41]:
df.groupby('process').count().orderBy('count', ascending=False).show()

+--------------------+--------+
|             process|   count|
+--------------------+--------+
|             Missing|15901328|
| Clinical evaluation|  513880|
|Routine surveillance|  189659|
|            Multiple|  128827|
| Laboratory reported|   82065|
|Contact tracing o...|   70802|
|             Unknown|   24163|
|               Other|    8776|
|Other detection m...|    8674|
|   Provider reported|    1892|
|Routine physical ...|     152|
|             Autopsy|       7|
+--------------------+--------+



In [42]:
df.groupby('exposure_yn').count().orderBy('count', ascending=False).show()

+-----------+--------+
|exposure_yn|   count|
+-----------+--------+
|    Missing|15034468|
|        Yes| 1438353|
|    Unknown|  457404|
+-----------+--------+



In [43]:
df.groupby('current_status').count().orderBy('count', ascending=False).show()

+--------------------+--------+
|      current_status|   count|
+--------------------+--------+
|Laboratory-confir...|15699194|
|       Probable Case| 1231031|
+--------------------+--------+



In [44]:
df.groupby('symptom_status').count().orderBy('count', ascending=False).show()

+--------------+-------+
|symptom_status|  count|
+--------------+-------+
|   Symptomatic|7519108|
|       Missing|6319041|
|       Unknown|2763959|
|  Asymptomatic| 328117|
+--------------+-------+



In [45]:
df.groupby('hosp_yn').count().orderBy('count', ascending=False).show()

+-------+-------+
|hosp_yn|  count|
+-------+-------+
|Missing|7044878|
|     No|6341619|
|Unknown|2622532|
|    Yes| 921196|
+-------+-------+



In [46]:
df.groupby('icu_yn').count().orderBy('count', ascending=False).show()

+-------+--------+
| icu_yn|   count|
+-------+--------+
|Missing|13606834|
|Unknown| 2076167|
|     No| 1154405|
|    Yes|   92819|
+-------+--------+



In [47]:
df.groupby('death_yn').count().orderBy('count', ascending=False).show()

+--------+-------+
|death_yn|  count|
+--------+-------+
|      No|8345229|
| Missing|5609568|
| Unknown|1879180|
|      NA| 865827|
|     Yes| 230421|
+--------+-------+



In [48]:
df.groupby('underlying_conditions_yn').count().orderBy('count', ascending=False).show()

+------------------------+--------+
|underlying_conditions_yn|   count|
+------------------------+--------+
|                    null|15642277|
|                     Yes| 1266461|
|                      No|   21487|
+------------------------+--------+



In [49]:
# Drop unnecessary columns based on number of values missing
df = df.drop('ethnicity', 'process', 'exposure_yn', 'current_status', 'symptom_status', 'hosp_yn', 'icu_yn', 'death_yn', 'underlying_conditions_yn')

In [50]:
df.show()

+---------+--------------+------+-------+
|res_state|     age_group|   sex|   race|
+---------+--------------+------+-------+
|       KY|  0 - 17 years|    NA|     NA|
|       GA|  0 - 17 years|Female|Unknown|
|       PA|  0 - 17 years|Female|     NA|
|       AR|  0 - 17 years|Female|     NA|
|       OK|  0 - 17 years|Female|     NA|
|       ME|  0 - 17 years|Female|     NA|
|       OR|  0 - 17 years|Female|     NA|
|       NC|  0 - 17 years|Female|     NA|
|       WI|  0 - 17 years|Female|     NA|
|       NC|  0 - 17 years|Female|     NA|
|       SC|18 to 49 years|Female|     NA|
|       OK|18 to 49 years|Female|     NA|
|       NC|18 to 49 years|Female|     NA|
|       TN|18 to 49 years|Female|Missing|
|       PA|18 to 49 years|Female|     NA|
|       MS|18 to 49 years|Female|     NA|
|       PA|18 to 49 years|Female|     NA|
|       GA|18 to 49 years|Female|Unknown|
|       TN|18 to 49 years|Female|Missing|
|       IA|50 to 64 years|Female|     NA|
+---------+--------------+------+-

In [51]:
# Replace "Missing" and "Unknown" with "NA"
df = df.replace(['Missing', 'Unknown'], 'NA')

In [52]:
df.show()

+---------+--------------+------+----+
|res_state|     age_group|   sex|race|
+---------+--------------+------+----+
|       KY|  0 - 17 years|    NA|  NA|
|       GA|  0 - 17 years|Female|  NA|
|       PA|  0 - 17 years|Female|  NA|
|       AR|  0 - 17 years|Female|  NA|
|       OK|  0 - 17 years|Female|  NA|
|       ME|  0 - 17 years|Female|  NA|
|       OR|  0 - 17 years|Female|  NA|
|       NC|  0 - 17 years|Female|  NA|
|       WI|  0 - 17 years|Female|  NA|
|       NC|  0 - 17 years|Female|  NA|
|       SC|18 to 49 years|Female|  NA|
|       OK|18 to 49 years|Female|  NA|
|       NC|18 to 49 years|Female|  NA|
|       TN|18 to 49 years|Female|  NA|
|       PA|18 to 49 years|Female|  NA|
|       MS|18 to 49 years|Female|  NA|
|       PA|18 to 49 years|Female|  NA|
|       GA|18 to 49 years|Female|  NA|
|       TN|18 to 49 years|Female|  NA|
|       IA|50 to 64 years|Female|  NA|
+---------+--------------+------+----+
only showing top 20 rows



In [53]:
# Filter for 'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO'
df_part1 = df.filter((df.res_state == ('AL' or 'AK' or 'AZ' or 'AR' or 'CA' or 'CO' or 'CT' or 'DE' or 'DC' or 'FL' or 'GA' or 'HI' or 'ID' or 'IL' or 'IN' or 'IA' or 'KS' or 'KY' or 'LA' or 'ME' or 'MD' or 'MA' or 'MI' or 'MN' or 'MS' or 'MO')))

In [54]:
# Filter for 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'
df_part2 = df.filter((df.res_state == ('MT' or 'NE' or 'NV' or 'NH' or 'NJ' or 'NM' or 'NY' or 'NC' or 'ND' or 'OH' or 'OK' or 'OR' or 'PA' or 'RI' or 'SC' or 'SD' or 'TN' or 'TX' or 'UT' or 'VT' or 'VA' or 'WA' or 'WV' or 'WI' or 'WY')))

In [55]:
df_part1.toPandas().to_csv('cdc_db_cleaned_part1.csv')

In [56]:
df_part2.toPandas().to_csv('cdc_db_cleaned_part2.csv')